In [1]:
!pip install -q pandas numpy scikit-learn transformers torch accelerate markdown


In [2]:
import pandas as pd
import numpy as np
import json

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from transformers import pipeline


Create the Dataset

In [3]:
data = {
    "customer_id":[1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015],
    "age":[25,45,35,62,29,41,50,22,38,47,33,55,28,60,40],
    "gender":["Male","Female","Male","Female","Female","Male","Male","Female","Male","Female","Male","Female","Male","Female","Male"],
    "income":[32000,78000,54000,91000,None,67000,120000,21000,59000,83000,51000,98000,35000,105000,69000],
    "credit_score":[650,720,680,800,610,700,850,580,690,740,670,820,640,830,710],
    "tenure_months":[12,48,24,60,6,36,72,3,30,54,18,66,9,70,40],
    "monthly_charges":[70.5,89.3,75,55.2,1200,85.7,40,95,78.2,62.5,82,50,110,45.5,88.9],
    "total_charges":[846,4286,1800,3312,7200,3085,2880,285,2346,3375,1476,3300,990,3185,3556],
    "contract_type":["Month-to-month","One year","Month-to-month","Two year","Month-to-month","One year","Two year","Month-to-month","One year","Two year","Month-to-month","Two year","Month-to-month","Two year","One year"],
    "payment_method":["Credit Card","Debit Card","UPI","Credit Card","Cash","Debit Card","Credit Card","UPI","Debit Card","Credit Card","UPI","Credit Card","Cash","Credit Card","Debit Card"],
    "internet_service":["Fiber","DSL","Fiber","DSL","Fiber","DSL","None","Fiber","DSL","DSL","Fiber","DSL","Fiber","None","DSL"],
    "num_complaints":[2,0,3,0,5,1,0,4,1,0,2,0,4,0,1],
    "last_login_days":[5,2,15,1,45,3,0,30,7,2,20,1,60,0,4],
    "avg_session_time":[35,60,20,80,10,55,90,15,50,75,25,85,12,95,65],
    "discount_rate":[0.1,0.05,0.15,0.02,0.25,0.08,0.01,0.2,0.07,0.03,0.18,0.02,0.3,0.01,0.06],
    "churn":[1,0,1,0,1,0,0,1,0,0,1,0,1,0,0]
}

df = pd.DataFrame(data)
df.to_csv("customer_churn_data.csv", index=False)
df.head()


,customer_id,age,gender,income,credit_score,tenure_months,monthly_charges,total_charges,contract_type,payment_method,internet_service,num_complaints,last_login_days,avg_session_time,discount_rate,churn
0,1001,25,Male,32000.0,650,12,70.5,846,Month-to-month,Credit Card,Fiber,2,5,35,0.10,1
1,1002,45,Female,78000.0,720,48,89.3,4286,One year,Debit Card,DSL,0,2,60,0.05,0
2,1003,35,Male,54000.0,680,24,75.0,1800,Month-to-month,UPI,Fiber,3,15,20,0.15,1
3,1004,62,Female,91000.0,800,60,55.2,3312,Two year,Credit Card,DSL,0,1,80,0.02,0
4,1005,29,Female,NaN,610,6,1200.0,7200,Month-to-month,Cash,Fiber,5,45,10,0.25,1


Dataset Summary Module

In [4]:
def dataset_summary(df):
    return {
        "shape": df.shape,
        "missing_percentage": df.isnull().mean() * 100,
        "dtypes": df.dtypes
    }

summary = dataset_summary(df)
summary


{'shape': (15, 16),
 'missing_percentage': customer_id         0.000000
 age                 0.000000
 gender              0.000000
 income              6.666667
 credit_score        0.000000
 tenure_months       0.000000
 monthly_charges     0.000000
 total_charges       0.000000
 contract_type       0.000000
 payment_method      0.000000
 internet_service    0.000000
 num_complaints      0.000000
 last_login_days     0.000000
 avg_session_time    0.000000
 discount_rate       0.000000
 churn               0.000000
 dtype: float64,
 'dtypes': customer_id           int64
 age                   int64
 gender               object
 income              float64
 credit_score          int64
 tenure_months         int64
 monthly_charges     float64
 total_charges         int64
 contract_type        object
 payment_method       object
 internet_service     object
 num_complaints        int64
 last_login_days       int64
 avg_session_time      int64
 discount_rate       float64
 churn          

**Automated Issue Detection**

1. Data Type Issues

In [5]:
def detect_type_issues(df):
    issues = []
    for col in df.columns:
        if df[col].dtype == "object":
            try:
                pd.to_numeric(df[col])
                issues.append(f"{col} stored as object but contains numeric values")
            except:
                pass
    return issues


2. Outlier Detection (ML-based)

In [6]:
def detect_outliers(df):
    num_df = df.select_dtypes(include=np.number).dropna()
    scaler = StandardScaler()
    X = scaler.fit_transform(num_df)

    iso = IsolationForest(contamination=0.05, random_state=42)
    preds = iso.fit_predict(X)

    if -1 in preds:
        return ["ML-based outliers detected in numerical features"]
    return []


3. Correlation Issues

In [7]:
def detect_correlations(df):
    corr = df.select_dtypes(include=np.number).corr()
    issues = []
    for i in range(len(corr.columns)):
        for j in range(i):
            if abs(corr.iloc[i, j]) > 0.9:
                issues.append(f"High correlation between {corr.columns[i]} and {corr.columns[j]}")
    return issues


4. Class Imbalance

In [8]:
def detect_class_imbalance(df, target):
    dist = df[target].value_counts(normalize=True)
    if dist.min() < 0.2:
        return ["Class imbalance detected in target variable"]
    return []


Load Open-Source LLM

In [15]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import warnings
import torch

warnings.filterwarnings("ignore")

model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 🔑 Fix: Explicitly set pad_token_id
tokenizer.pad_token = tokenizer.eos_token

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id
)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


AI Explanation

In [25]:
def explain_issue(issue):
    prompt = f"""
Explain clearly in 4–5 sentences:
1. Why this data quality issue matters
2. How it impacts ML model performance

Issue: {issue}
Answer:
"""
    response = llm(
        prompt,
        max_new_tokens=120,
        do_sample=False
    )[0]["generated_text"]

    return response.split("Answer:")[-1].strip()


AI Recommendations

In [26]:
def recommend_fix(issue):
    prompt = f"""
Give 3 concise, professional fixes for this data issue:

Issue: {issue}
Fixes:
"""
    response = llm(
        prompt,
        max_new_tokens=120,
        do_sample=False
    )[0]["generated_text"]

    return response.split("Fixes:")[-1].strip()


In [28]:
issues = []
issues += detect_type_issues(df)
issues += detect_outliers(df)
issues += detect_correlations(df)
issues += detect_class_imbalance(df, "churn")

audit_results = []

for issue in issues:
    audit_results.append({
        "issue": issue,
        "explanation": explain_issue(issue),
        "recommendation": recommend_fix(issue)
    })

audit_results


[{'issue': 'ML-based outliers detected in numerical features',
  'explanation': '1. This data quality issue matters because it can significantly impact the performance of machine learning models. Outliers are data points that deviate significantly from the normal pattern or distribution of the data. If these outliers are not properly handled, they can lead to biased or inaccurate predictions by the ML models. This can result in poor decision-making and suboptimal outcomes.\n2. Outliers in numerical features can have a significant impact on ML model performance. They can skew the distribution of the data, leading to biased estimations of model parameters. This can result in overfitting or under',
  'recommendation': '1. Remove outliers using statistical methods such as the Z-score.\n2. Winsorize the data by replacing extreme values with less extreme values.\n3. Use robust regression algorithms that are less sensitive to outliers.'},
 {'issue': 'High correlation between income and age',


Define Risk Score Function

In [38]:
def calculate_dataset_risk_score(summary, audit_results):
    risk_score = 0

    # 1️⃣ Missing value risk (max 30)
    missing_percent = summary["missing_percentage"].mean()
    risk_score += min(missing_percent * 2, 30)

    # 2️⃣ Issue count risk (max 40)
    issue_count = len(audit_results)
    risk_score += min(issue_count * 3, 40)

    # 3️⃣ Severity-based risk (max 30)
    for item in audit_results:
        issue = item["issue"].lower()
        if "outlier" in issue:
            risk_score += 10
        if "correlation" in issue:
            risk_score += 5

    # Cap score at 100
    return min(int(risk_score), 100)


In [39]:
dataset_risk_score = calculate_dataset_risk_score(summary, audit_results)
dataset_risk_score


100

In [40]:
def interpret_risk(score):
    if score < 30:
        return "Low Risk – Dataset is mostly clean and reliable."
    elif score < 60:
        return "Medium Risk – Data cleaning and validation recommended."
    else:
        return "High Risk – Significant data quality issues detected."


In [41]:
interpretation = interpret_risk(dataset_risk_score)
interpretation


'High Risk – Significant data quality issues detected.'

Final Audit Report

In [42]:
def generate_report(summary, audit_results, risk_score, interpretation):
    report = "# AI Data Quality Audit Report\n\n"

    report += "## Dataset Overview\n"
    report += f"- Shape: {summary['shape']}\n"
    report += f"- Dataset Risk Score: **{risk_score}/100**\n"
    report += f"- Risk Interpretation: **{interpretation}**\n\n"

    report += "## Detected Issues & AI Reasoning\n\n"

    for idx, item in enumerate(audit_results, start=1):
        report += f"### {idx}. {item['issue']}\n\n"
        report += f"**(a) Why it matters:**\n{item['explanation']}\n\n"
        report += f"**(b) Recommendation:**\n{item['recommendation']}\n\n"
        report += "---\n\n"

    return report


In [43]:
final_report = generate_report(
    summary,
    audit_results,
    dataset_risk_score,
    interpretation
)

print(final_report)


# AI Data Quality Audit Report

## Dataset Overview
- Shape: (15, 16)
- Dataset Risk Score: **100/100**
- Risk Interpretation: **High Risk – Significant data quality issues detected.**

## Detected Issues & AI Reasoning

### 1. ML-based outliers detected in numerical features

**(a) Why it matters:**
1. This data quality issue matters because it can significantly impact the performance of machine learning models. Outliers are data points that deviate significantly from the normal pattern or distribution of the data. If these outliers are not properly handled, they can lead to biased or inaccurate predictions by the ML models. This can result in poor decision-making and suboptimal outcomes.
2. Outliers in numerical features can have a significant impact on ML model performance. They can skew the distribution of the data, leading to biased estimations of model parameters. This can result in overfitting or under

**(b) Recommendation:**
1. Remove outliers using statistical methods such as

In [46]:
# Create README.md with natural-language content

readme_content = """
# Autonomous AI System for Data Quality Auditing

## Overview
This project implements an **Autonomous AI-powered Data Quality Auditor** that automatically analyzes datasets, identifies data quality problems, explains why those problems matter, and suggests corrective actions.

The system is designed to behave like a **junior data analyst**, capable of auditing datasets without human guidance and producing a professional, human-readable audit report.

---

## Problem Statement
Real-world datasets often contain hidden quality issues such as:
- Missing or inconsistent values
- Outliers that silently affect model performance
- Highly correlated or redundant features
- Data patterns that can bias machine learning models

These issues are often overlooked and can significantly degrade the reliability of analytics and ML systems.
This project addresses that problem by building an **AI-driven automated auditing pipeline**.

---

## Objectives
The main objectives of this project are:
- Automatically analyze any CSV dataset
- Detect common and advanced data quality issues
- Use an AI model to **reason** about why each issue matters
- Recommend practical data preprocessing and modeling fixes
- Generate a structured audit report
- Compute an overall **Dataset Risk Score (0–100)**

---

## Input Requirements
- Dataset format: CSV
- Minimum of **15 columns**
- At least **1 target variable**
- Works on unseen datasets without hardcoded rules

---

## Output
The system generates the following outputs:
- Dataset summary (shape, data types, missing value percentages)
- Detected data quality issues (outliers, correlations, etc.)
- AI-generated explanations describing the impact on ML models
- AI-generated recommendations for fixing each issue
- Dataset Risk Score with interpretation
- Final audit report in Markdown format

---

## Dataset Risk Score
The Dataset Risk Score provides a single numerical measure of data quality risk.

- **0–30** → Low Risk (dataset is mostly clean)
- **31–60** → Medium Risk (cleaning and validation recommended)
- **61–100** → High Risk (significant data quality issues present)

The score is computed based on missing values, number of detected issues, and their severity.

---

## Technology Stack
- Python
- Pandas, NumPy
- Scikit-learn
- HuggingFace Transformers
- Open-source Large Language Model: **Microsoft Phi-2**

---

## Project Structure
AI-Data-Quality-Auditor/
├── data/               # Sample dataset
├── notebooks/          # Google Colab / Jupyter notebook
├── src/                # Modular source code
├── reports/            # Generated audit reports
├── requirements.txt    # Python dependencies
└── README.md           # Project documentation

---

## How to Run the Project

### Option 1: Google Colab (Recommended)
1. Open the notebook `AI_Data_Quality_Auditor.ipynb` in Google Colab
2. Install dependencies using the provided cells
3. Upload or use the sample CSV dataset
4. Run all cells from top to bottom
5. View the generated audit report and risk score

### Option 2: Local Execution
1. Clone the repository
2. Install dependencies using `pip install -r requirements.txt`
3. Run the notebook or Python scripts
4. Review the generated audit report

---

## How the System Works
1. Loads the dataset
2. Generates a dataset summary
3. Detects data quality issues automatically
4. Uses an AI model to explain why each issue matters
5. Suggests fixes and improvements
6. Calculates dataset risk score
7. Produces a final audit report

---

## Future Improvements
- Auto-fix mode with before/after comparison
- Streamlit-based user interface
- PDF export of audit reports
- Feature importance and validation diagnostics

---

## Conclusion
This project demonstrates how **agentic AI systems** can be used to automate data quality auditing.
By combining statistical analysis with AI-based reasoning, the system provides meaningful insights that help improve dataset reliability and machine learning performance.
"""

# Write README.md
with open("README.md", "w") as f:
    f.write(readme_content)

print("✅ README.md created successfully!")

# Download README.md
from google.colab import files
files.download("README.md")


✅ README.md created successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>